In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from canopy_app import config_cases, run, run_config_sens, DEFAULT_POINT_INPUT

xr.set_options(display_expand_data=False, display_expand_attrs=False)

plt.rcParams.update(
    {
        "figure.autolayout": True,
        "axes.formatter.limits": (-3, 4),
        "axes.formatter.use_mathtext": True,
        "figure.max_open_warning": 0,
    }
)

%matplotlib widget

## Default case (2-D)

Southeast US

```
provide spatial plots for 3D vars (e.g., ave. in-canopy ws, kz, rjcf and sum in-canopy biogenic isoprene) and 2D vars (flame height, waf)
```

In [ ]:
inp = xr.open_dataset("../input/gfs.t12z.20220701.sfcf000.canopy.nc")

In [ ]:
inp

In [ ]:
%%time

ds = run()

In [ ]:
ds.waf.plot(x="lon", y="lat", size=3)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(projection=proj)

ax.coastlines()
ax.gridlines(draw_labels=True)

ds.waf.plot(x="lon", y="lat", ax=ax, transform=tran, alpha=0.8)

## Canopy thresholds

```
Maybe, lai_thresh = 0.1
frt_thresh = 0.1
fch_thresh = 0.5...
and
lai_thresh = 0.5
frt_thresh = 0.5
fch_thresh = 0.5
or,
lai_thresh = 0.1
frt_thresh = 0.5
fch_thresh = 10.0 (or 3.0) or something only representing tall forest stands...
```

## PAI options

```
For, flameh_opt/flameh_set,/frp_fac, here is a suggestion of case sensitivities:
0, 1.0, 1.0  (Straight FRP to flameh calculation, only for active FRP/fire points, no FRP tuning)
1, 2.0, 1.0  (User-set flame height = 2.0 m for all canopy points)
2, 2.0, 1.0  (FRP to flameh calculation for active FRP/fire points,   user-set flame height = 2.0 m elsewhere, no FRP tuning)
3, 0.5, 1.0  (Flameh is set to 0.5*Hc (m) for all canopy points)
4, 0.5, 1.0  (FRP to flameh calculation for active FRP/fire points, flameh is set to 0.5*Hc (m) elsewhere, no FRP tuning)
5, 0.5, 10.0  (FRP to flameh calculation for active FRP/fire points with crowning dependence (flameh=Hc), flameh is set to 0.5*Hc (m) elsewhere, FRP increased by a factor of 10)
```

## Flame height options

```
For pai_opt/pai_set, here is a list of sensitivities:
0, 1 (PAI from Katul et al., 2004 vegetation types/lookup table)
2, 1 (PAI estimated from model LAI)
3, 1 (User-set PAI = 1)
3, 2 (User-set PAI = 2)
3, 4 (User-set PAI = 4)
```

## Canopy environment coefficient (biogenics)

```
Also, we should have a sensitivity for the biogenics, using the bio_cce (canopy environment coefficient) that is a tuning parameter for different model inputs..(the default 0.21 is based on Silva et al. and GEOS-Chem environment)...
```

## Default point case

In [ ]:
DEFAULT_POINT_INPUT.T.rename(columns={0: "value"})

In [ ]:
%%time
ds = run(
    config={
        "filenames": {"file_vars": "../input/input_variables_point.txt"},
        "userdefs": {"infmt_opt": 1, "nlat": 1, "nlon": 1},
    },
)

In [ ]:
vns = ["ws", "kz", "rjcf", "emi_isop"]

fig, axs = plt.subplots(1, len(vns), figsize=(10, 4), sharey=True)

for vn, ax in zip(vns, axs.flat):
    ax.axhline(ds.canopy_height, c="0.5", ls="--")
    ds[vn].plot(y="z", ax=ax)

ax.set_ylim(ymin=0);

In [ ]:
(
    ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]]
    .sel(z=10, method="nearest")
    .to_pandas()
    .to_frame()
    .plot.bar(figsize=(6, 3), legend=False)
)
plt.yscale("log")

## $z_0 / h_c$

The namelist parameter `z0ghc` represents the ratio of ground roughness length to canopy top height, i.e.
$z_0 / h_c$. In general, they increase together, but this is still a tunable parameter.

In [ ]:
%%time

cases = config_cases(
    file_vars="../input/input_variables_point.txt",
    infmt_opt=1,
    nlat=1,
    nlon=1,
    z0ghc=np.power(10, np.linspace(-4, np.log10(0.25), 100)).tolist(),
    product=False,
)
ds = run_config_sens(cases)
ds

In [ ]:
hc = ds.canopy_height.isel(case=0)
assert (ds.canopy_height == hc).all()
ds_c = ds.isel(z=ds.z <= hc).mean("z")  # canopy mean

vns = [
    "ws",
    "kz",
    "waf",
]

fig, axs = plt.subplots(1, len(vns), figsize=(9, 3.5), sharey=False)

fig.suptitle("Canopy mean")

for vn, ax in zip(vns, axs.flat):
    ax.plot(ds_c["z0ghc"], ds_c[vn])
    ax.set(ylabel=vn, xlabel="$z_0 / h_c$")
    ax.set_xscale("log")

## Input variables

We can create different point input files based on the default case and run the model for each,
allowing us to examine the sensitivity to the input variables.

Note that files matching `test_*.txt` are gitignored.